<a href="https://colab.research.google.com/github/tshiven/Bipedal-Walker/blob/main/Bipedal_Walker_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 1. Install the system dependency 'swig' using Linux commands
!sudo apt-get update && sudo apt-get install -y swig

# 2. Install the Python libraries
!pip install gymnasium[box2d] stable-baselines3 shimmy moviepy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:

In [5]:
pip install gymnasium[box2d] stable-baselines3 shimmy

In [6]:
import gymnasium as gym
from stable_baselines3 import PPO
import os

# 1. Setup Environment
# 'render_mode' is None during training for speed, 'human' for watching it.
env_id = "BipedalWalker-v3"

def train():
    env = gym.make(env_id)

    # 2. Initialize the Agent
    # We use PPO with a standard MLP policy.
    # Note: BipedalWalker-v3 is a continuous task, PPO handles this naturally.
    model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_bipedal_logs/")

    # 3. Train the Agent
    # For a bipedal walker to actually walk, you usually need ~500,000 to 1M steps.
    print("Starting training...")
    model.learn(total_timesteps=100000)

    # 4. Save the model
    model.save("ppo_bipedal_walker")
    print("Model saved.")
    env.close()

def render_ai():
    # 5. Load and Render
    print("Loading model for rendering...")
    env = gym.make(env_id, render_mode="human")
    model = PPO.load("ppo_bipedal_walker")

    obs, info = env.reset()
    for _ in range(1000):
        # Predict the action based on the observation
        # deterministic=True is usually better for evaluation
        action, _states = model.predict(obs, deterministic=True)

        # Apply the action to the environment
        obs, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated or truncated:
            obs, info = env.reset()

    env.close()

if __name__ == "__main__":
    # Toggle these based on what you want to do
    train()
    render_ai()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: De

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Starting training...
Logging to ./ppo_bipedal_logs/PPO_1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 580      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    fps             | 1021     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 510          |
|    ep_rew_mean          | -110         |
| time/                   |              |
|    fps                  | 748          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0071190856 |
|    clip_fraction        | 0.0722       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.68        |
|    explained_variance   | -0.0121      |
|    learning_r

In [9]:
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordVideo
from IPython.display import HTML
from base64 import b64encode
import os

# --- 1. SETUP PATHS ---
# We force the creation of a video folder so it definitely exists
video_folder = "./video_output"
os.makedirs(video_folder, exist_ok=True)

# Check where your model is.
# If you saved it as "ppo_bipedal_walker", it's likely in the current directory.
model_path = "ppo_bipedal_walker"

# --- 2. RECORD THE VIDEO ---
print("Setting up environment for recording...")
env = gym.make("BipedalWalker-v3", render_mode="rgb_array")

# This wrapper records the video and puts it in the folder we just created
env = RecordVideo(env, video_folder=video_folder, name_prefix="bipedal_test")

print(f"Loading model from: {model_path}")
# Load the model (try/except block handles if you used a different name)
try:
    model = PPO.load(model_path)
except FileNotFoundError:
    # Fallback: try looking in a 'models' folder just in case
    model = PPO.load(f"models/{model_path}")

obs, info = env.reset()
done = False

print("Recording 1 episode...")
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()

# --- 3. WATCH THE VIDEO ---
# Find the video file we just made
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
if len(video_files) > 0:
    mp4_path = os.path.join(video_folder, video_files[-1])
    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    print(f"Success! Displaying video...")
    display(HTML(f"""
    <video width=400 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """))
else:
    print("Error: Video file was not created.")

Setting up environment for recording...


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/video_output folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Loading model from: ppo_bipedal_walker
Recording 1 episode...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Success! Displaying video...
